# CRPBot Model Evaluation - FIXED

**Models**: 3 LSTM (BTC, ETH, SOL) - 128/3/bidirectional, 50 features  
**Goal**: Evaluate against promotion gates (≥68% accuracy, ≤5% calibration error)  
**Time**: ~5-10 minutes on GPU

In [ ]:
# Step 1: Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Step 2: Install dependencies
!pip install -q loguru pandas pyarrow scikit-learn

In [ ]:
# Step 3: Mount Drive & verify
from google.colab import drive
drive.mount('/content/drive')

import os
path = '/content/drive/MyDrive/crpbot'
print(f"\nPath exists: {os.path.exists(path)}")
if os.path.exists(path):
    !ls -lh /content/drive/MyDrive/crpbot/

In [ ]:
# Step 4: Copy files
!mkdir -p models/new data/features
!cp /content/drive/MyDrive/crpbot/models/*.pt models/new/ 2>/dev/null || cp /content/drive/MyDrive/crpbot/*.pt models/new/
!cp /content/drive/MyDrive/crpbot/features/*.parquet data/features/ 2>/dev/null || cp /content/drive/MyDrive/crpbot/*.parquet data/features/

import glob
print(f"\nModels: {len(glob.glob('models/new/*.pt'))}/3")
print(f"Features: {len(glob.glob('data/features/*.parquet'))}/3")

In [ ]:
# Step 5: Setup modules
!mkdir -p apps/trainer/{models,train}
!touch apps/__init__.py apps/trainer/__init__.py apps/trainer/models/__init__.py apps/trainer/train/__init__.py

In [ ]:
%%writefile apps/trainer/models/lstm.py
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    """LSTM matching checkpoint structure."""
    def __init__(self, input_size: int, hidden_size: int = 128, num_layers: int = 3, dropout: float = 0.2, bidirectional: bool = True):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0, bidirectional=bidirectional)
        lstm_out = hidden_size * 2 if bidirectional else hidden_size
        # Match checkpoint structure: fc.0, fc.3
        self.fc = nn.Sequential(
            nn.Linear(lstm_out, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [ ]:
%%writefile apps/trainer/data_pipeline.py
from pathlib import Path
import pandas as pd
from loguru import logger

def load_features_from_parquet(symbol: str):
    patterns = [f"features_{symbol}_*.parquet"]
    for p in patterns:
        files = list(Path("data/features").glob(p))
        if files:
            df = pd.read_parquet(sorted(files)[-1])
            logger.info(f"Loaded: {len(df):,} rows, {len(df.columns)} cols")
            return df
    raise FileNotFoundError(f"No features for {symbol}")

def create_walk_forward_splits(df, train_ratio=0.7, val_ratio=0.15):
    n = len(df)
    t1, t2 = int(n * train_ratio), int(n * (train_ratio + val_ratio))
    logger.info(f"Splits: Train={t1}, Val={t2-t1}, Test={n-t2}")
    return df.iloc[:t1].copy(), df.iloc[t1:t2].copy(), df.iloc[t2:].copy()

In [ ]:
%%writefile apps/trainer/features.py
import pandas as pd
from loguru import logger

# All 50 features
FEATURE_COLUMNS = [
    "session_tokyo", "session_london", "session_ny", "day_of_week", "is_weekend",
    "spread", "spread_pct", "atr", "spread_atr_ratio",
    "volume_ma", "volume_ratio", "volume_trend",
    "sma_7", "sma_14", "sma_21", "sma_50", "price_to_sma_7", "price_to_sma_14", "price_to_sma_21", "price_to_sma_50",
    "rsi", "macd", "macd_signal", "macd_diff", "bb_upper", "bb_middle", "bb_lower", "bb_width",
    "5m_open", "5m_high", "5m_low", "5m_close", "5m_volume",
    "15m_open", "15m_high", "15m_low", "15m_close", "15m_volume",
    "1h_open", "1h_high", "1h_low", "1h_close", "1h_volume",
    "cross_tf_alignment_score", "cross_tf_alignment_direction", "cross_tf_alignment_strength",
    "atr_percentile",
]

def normalize_features(train_df, val_df=None, test_df=None):
    cols = [c for c in FEATURE_COLUMNS if c in train_df.columns]
    logger.info(f"Found {len(cols)}/50 features: {cols[:5]}...")
    stats = {"mean": train_df[cols].mean().to_dict(), "std": train_df[cols].std().to_dict()}
    for col in cols:
        m, s = stats["mean"][col], stats["std"][col]
        if s > 0:
            train_df[col] = (train_df[col] - m) / s
            if val_df is not None: val_df[col] = (val_df[col] - m) / s
            if test_df is not None: test_df[col] = (test_df[col] - m) / s
    return train_df, val_df, test_df, stats, cols

In [ ]:
%%writefile apps/trainer/train/dataset.py
import numpy as np
import torch
from torch.utils.data import Dataset
from loguru import logger

class SequenceDataset(Dataset):
    def __init__(self, df, feature_columns, seq_len=60, pred_horizon=15):
        self.features = df[feature_columns].values.astype(np.float32)
        future = df["close"].shift(-pred_horizon)
        self.targets = (future > df["close"]).astype(np.float32).values
        self.valid = np.arange(seq_len, len(df) - pred_horizon)
        self.seq_len = seq_len
        logger.info(f"Dataset: {len(self.valid)} seqs, {len(feature_columns)} feats")
    
    def __len__(self): return len(self.valid)
    
    def __getitem__(self, idx):
        i = self.valid[idx]
        return torch.from_numpy(self.features[i-self.seq_len:i]), torch.tensor([self.targets[i]], dtype=torch.float32)

In [ ]:
%%writefile evaluate.py
import numpy as np
import pandas as pd
import torch
from loguru import logger
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader
from apps.trainer.models.lstm import LSTMModel
from apps.trainer.data_pipeline import load_features_from_parquet, create_walk_forward_splits
from apps.trainer.features import normalize_features
from apps.trainer.train.dataset import SequenceDataset

def evaluate_model(model_path, symbol, device="cuda"):
    logger.info(f"\n{'='*60}\n🚀 {symbol}\n{'='*60}")
    df = load_features_from_parquet(symbol)
    train_df, val_df, test_df = create_walk_forward_splits(df)
    train_df, val_df, test_df, stats, feature_cols = normalize_features(train_df, val_df, test_df)
    
    # Critical: use actual number of features found
    n_features = len(feature_cols)
    logger.info(f"Creating model with {n_features} features")
    
    test_dataset = SequenceDataset(test_df, feature_cols)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, pin_memory=(device=="cuda"))
    
    model = LSTMModel(input_size=n_features, hidden_size=128, num_layers=3, dropout=0.2, bidirectional=True)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model = model.to(device).eval()
    
    logger.info(f"Evaluating {len(test_dataset):,} sequences...")
    all_preds, all_targets, all_probs = [], [], []
    
    with torch.no_grad():
        for batch_idx, (seqs, tgts) in enumerate(test_loader):
            outs = model(seqs.to(device)).squeeze().cpu().numpy()
            probs = outs if outs.ndim > 0 else [outs.item()]
            preds = (np.array(probs) >= 0.5).astype(int)
            all_probs.extend(probs.tolist() if isinstance(probs, np.ndarray) else [probs])
            all_preds.extend(preds.tolist())
            all_targets.extend(tgts.squeeze().numpy().tolist())
            if batch_idx % 100 == 0:
                logger.info(f"  {len(all_preds):,}/{len(test_dataset):,}")
    
    acc = accuracy_score(all_targets, all_preds)
    prec = precision_score(all_targets, all_preds, zero_division=0)
    rec = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds, zero_division=0)
    
    # Calibration
    bins = np.linspace(0, 1, 11)
    bin_idx = np.digitize(all_probs, bins) - 1
    ece = sum(
        (mask := bin_idx == i).sum() / len(all_targets) * abs(
            np.array(all_targets)[mask].mean() - np.array(all_probs)[mask].mean()
        ) for i in range(10) if mask.sum() > 0
    )
    
    logger.info(f"\n✅ Results: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, ECE={ece:.4f}")
    passes = acc >= 0.68 and ece <= 0.05
    logger.info(f"🎯 Acc≥68% {'✅' if acc>=0.68 else '❌'} ({acc:.2%}), Cal≤5% {'✅' if ece<=0.05 else '❌'} ({ece:.2%})")
    logger.info(f"{'✅ PASS' if passes else '❌ FAIL'}\n")
    
    return {"symbol": symbol, "accuracy": acc, "precision": prec, "recall": rec, "f1": f1, "calibration_error": ece, "samples": len(all_targets), "features_used": n_features}

device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Device: {device}")
results = {}
for symbol in ["BTC-USD", "ETH-USD", "SOL-USD"]:
    try:
        model_path = f"models/new/lstm_{symbol.replace('-', '_')}_1m_7b5f0829.pt"
        results[symbol] = evaluate_model(model_path, symbol, device)
    except Exception as e:
        logger.error(f"Failed {symbol}: {e}")
        import traceback
        traceback.print_exc()

if results:
    df = pd.DataFrame(results).T
    print(f"\n{'='*60}\n📊 SUMMARY\n{'='*60}\n{df.to_string()}\n")
    df.to_csv("results.csv")
    logger.info("✅ Saved results.csv")
else:
    logger.error("No results - all evaluations failed")

In [ ]:
# Step 6: Run evaluation
!python evaluate.py

In [ ]:
# Step 7: Download results
from google.colab import files
files.download('results.csv')
print("\n✅ Share results.csv in Claude chat!")